
# 19x19 Coupler


Importing the script dependencies



In [ ]:
from SuPyMode.workflow import Workflow, configuration, fiber_catalogue, Boundaries
import numpy

## Creating the fiber list for mesh
In this example we want to simulate a single fiber at wavelength 1550 nm.



In [ ]:
wavelength = 1550e-9

fiber_list = []

## Generating the fiber structure
Here we define the cladding and fiber structure to model the problem



In [ ]:
clad_structure = configuration.ring.FusedProfile_19x19

for radius in numpy.linspace(10e-6, 50e-6, 19):
    fiber = fiber_catalogue.GenericFiber(wavelength=wavelength)
    fiber.add_silica_pure_cladding()
    fiber.create_and_add_new_structure(name='core', radius=radius / 2, NA=0.115)

    fiber_list.append(fiber)

capillary_tube = fiber_catalogue.CapillaryTube(
    radius=350e-6,
    wavelength=wavelength,
    delta_n=-15e-3
)

Defining the boundaries of the system



In [ ]:
boundaries = [
    Boundaries(right='symmetric', bottom='symmetric'),
    Boundaries(right='symmetric', bottom='anti-symmetric')
]

## Generating the computing workflow
Workflow class to define all the computation parameters before initializing the solver



In [ ]:
workflow = Workflow(
    fiber_list=fiber_list,          # List of fiber to be added in the mesh, the order matters.
    capillary_tube=capillary_tube,  # Adding additional capillariy tube structure
    clad_structure=clad_structure,  # Cladding structure, if None provided then no cladding is set.
    fusion_degree='auto',           # Degree of fusion of the structure if applicable.
    wavelength=wavelength,          # Wavelength used for the mode computation.
    resolution=60,                  # Number of point in the x and y axis [is divided by half if symmetric or anti-symmetric boundaries].
    x_bounds="left",                # Mesh x-boundary structure.
    y_bounds="top",                 # Mesh y-boundary structure.
    boundaries=boundaries,          # Set of symmetries to be evaluated, each symmetry add a round of simulation
    n_sorted_mode=3,                # Total computed and sorted mode.
    n_added_mode=3,                 # Additional computed mode that are not considered later except for field comparison [the higher the better but the slower].
    plot_geometry=True,             # Plot the geometry mesh before computation.
    debug_mode=0,                   # Print the iteration step for the solver plus some other important steps.
    auto_label=True,                # Auto labeling the mode. Label are not always correct and should be verified afterwards.
    itr_final=0.1,                  # Final value of inverse taper ratio to simulate
    clad_rotation=0,                # Rotate the geoemtry in the given angle in degree
    index_scrambling=1e-4           # Scrambling of refractive index value in order to lift mode degeneracy [useful for some analysis]
)

superset = workflow.get_superset()

## Field computation: $E_{i,j}$



In [ ]:
_ = superset.plot(plot_type='field', itr_list=[1.0, 0.1])

## Effective index: $n^{eff}_{i,j}$



In [ ]:
_ = superset.plot(plot_type='index')

## Modal normalized coupling: $C_{i,j}$



In [ ]:
_ = superset.plot(plot_type='normalized-coupling')

## Adiabatic criterion: $\tilde{C}_{i,j}$



In [ ]:
_ = superset.plot(plot_type='adiabatic')

# -